In [19]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import torch
import math
from torch.utils.tensorboard import SummaryWriter # noqa

from rl_envs.gym_grid_world_env import GridWorldEnv
from agents.policy_gradient import PGAgent
from tools.helper import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
LEARN_RATE = 0.001
DISCOUNTED_FACTOR = 0.9

FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 1

In [21]:
env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
agent = PGAgent(2, env.action_n, lr = LEARN_RATE, discounted_factor=DISCOUNTED_FACTOR)
writer = SummaryWriter()

In [22]:
episode_rewards = []
episode_lengths = []


In [27]:
num_episodes = 1000
episode_len = 100
iter_counter = 0
for episode in range(num_episodes):
    # 首先, 根据 policy 生成 episode
    state = (0,0)
    trajectory = []
    obs, _ = env.reset()
    for _ in range(episode_len):
        state = tuple(obs['agent'])
        action = agent.get_action(state)
        obs, reward, terminated , truncated, info = env.step(action)
        # next_state = tuple(obs['agent'])
        trajectory.append((state, action, reward))
        state = obs

    # 然后, 根据生成的 episode, 对 Q value 和 policy 进行更新
    for _ in range(10):
        last_loss = agent.update(trajectory)  
    # iter_counter+=1
    writer.add_scalar('Loss', last_loss, episode)



In [28]:
policy = agent.generate_policy_table(env.height, env.width)

print_by_dict(env, policy)

for i in range(env.height):
    print("[", end=" ")
    for j in range(env.width):
        state = (i,j)
        action = np.argmax(policy[state])
        print(env.action_mappings[action], end=" ")
    print("]")

[ [[0.04699978 0.8079529  0.0485365  0.04845719 0.04805375]] [[0.00263254 0.988766   0.00266724 0.00296175 0.00297244]] [[2.6110813e-05 9.9988234e-01 2.5975356e-05 3.2191856e-05 3.3329899e-05]] [[1.9309658e-07 9.9999905e-01 1.8518118e-07 2.6153560e-07 2.7853201e-07]] [[1.3337742e-09 1.0000000e+00 1.2251686e-09 1.9816628e-09 2.1631956e-09]] ]
[ [[0.0083405  0.96237487 0.00936534 0.01016512 0.0097541 ]] [[4.0088277e-04 9.9818844e-01 4.1187386e-04 5.1616534e-04 4.8258348e-04]] [[6.98770737e-06 9.99966502e-01 6.97672021e-06 1.00511925e-05
  9.39305755e-06]] [[6.2289494e-08 9.9999952e-01 6.0441948e-08 9.8219672e-08 9.5578208e-08]] [[4.5959511e-10 1.0000000e+00 4.3028878e-10 8.0313584e-10 7.9973317e-10]] ]
[ [[3.8316238e-04 9.9809819e-01 4.5520626e-04 5.6154694e-04 5.0185708e-04]] [[1.7937997e-05 9.9990988e-01 1.9425403e-05 2.8188530e-05 2.4647090e-05]] [[5.3177712e-07 9.9999726e-01 5.2927464e-07 9.0034462e-07 7.7707955e-07]] [[8.7590797e-09 1.0000000e+00 8.5347915e-09 1.6744481e-08 1.437677

In [ ]:
gridworld_demo(agent, forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)